In [2]:
!pip install streamlit pyngrok matplotlib numpy -q

In [7]:
%%writefile app.py

import streamlit as st
st.set_page_config(page_title="Self‑Parking AI Simulator", layout="centered")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

st.title("🚗 Automated Valet Parking Path Planner")

# ---------------- Step 1: User selects parking strategy ----------------
strategy = st.selectbox(
    "Select Parking Strategy",
    ["Forward Perpendicular", "Reverse Parallel", "Cannot Park"]
)

# ---------------- Step 2: User inputs ----------------
spot_length = st.number_input("Spot Length (m)", value=6.2)
spot_width  = st.number_input("Spot Width (m)", value=3.0)
car_length  = st.number_input("Car Length (m)", value=spot_length * 0.6)
car_width   = st.number_input("Car Width (m)", value=spot_width * 0.55)
distance_to_spot = st.number_input("Distance to Spot (m)", value=5.0)

spot_origin = np.array([15.0, 3.0])

# ================== Helpers (mini extract) ==================
def generate_bezier_curve(p0, p1, p2, n=40):
    """Quadratic Bézier, returns (n,2) points."""
    t = np.linspace(0, 1, n).reshape(-1, 1)
    p0, p1, p2 = map(np.array, (p0, p1, p2))
    return (1 - t)**2 * p0 + 2*(1 - t)*t*p1 + t**2 * p2

def sample_bezier(p0, p1, p2, p3, n=50):
    """Cubic Bézier, returns list[(x,y)] for S-curve reverse."""
    t = np.linspace(0.0, 1.0, n)[:, None]
    p0 = np.asarray(p0, float); p1 = np.asarray(p1, float)
    p2 = np.asarray(p2, float); p3 = np.asarray(p3, float)
    B = (1-t)**3*p0 + 3*(1-t)**2*t*p1 + 3*(1-t)*t**2*p2 + t**3*p3
    return [(float(x), float(y)) for x, y in B]

def make_arc(start_xy, mid_offset, end_offset, n=40):
    """Convenience wrapper to build a quadratic arc with relative offsets."""
    sx, sy = start_xy
    p0 = (sx, sy)
    p1 = (sx + mid_offset[0], sy + mid_offset[1])
    p2 = (sx + end_offset[0], sy + end_offset[1])
    return generate_bezier_curve(p0, p1, p2, n)

def straight_segment(start_xy, length, heading_deg, n=15):
    """Straight segment of length at heading_deg, (n,2) samples."""
    sx, sy = start_xy
    rad = np.radians(heading_deg)
    ex, ey = sx + length*np.cos(rad), sy + length*np.sin(rad)
    return np.column_stack([np.linspace(sx, ex, n),
                            np.linspace(sy, ey, n)])

def concat_paths(*paths):
    """Concatenate multiple path segments; skip seam duplicates."""
    out = []
    for k, p in enumerate(paths):
        p = np.asarray(p)
        if k > 0 and len(out) and len(p):
            out.extend(p[1:])  # skip duplicate seam point
        else:
            out.extend(p)
    return np.array(out)

def heading_at(path, i):
    """Heading (deg) at index i using finite differences."""
    if i == 0:
        p_prev, p_curr = path[i], path[i+1]
    else:
        p_prev, p_curr = path[i-1], path[i]
    dx = p_curr[0] - p_prev[0]
    dy = p_curr[1] - p_prev[1]
    return np.degrees(np.arctan2(dy, dx))

def rear_bumper_center(center_xy, yaw_deg, car_length):
    """Convert car center + yaw to rear bumper position."""
    rad = np.radians(yaw_deg)
    return np.array(center_xy) - 0.5 * car_length * np.array([np.cos(rad), np.sin(rad)])

def rotate_path_about_point(points, center_xy, delta_deg):
    """Rotate an array/list of (x,y) about center_xy by delta_deg degrees."""
    rad = np.radians(delta_deg)
    R = np.array([[np.cos(rad), -np.sin(rad)],
                  [np.sin(rad),  np.cos(rad)]])
    P = np.asarray(points) - center_xy
    P = P @ R.T
    P = P + center_xy
    return [tuple(p) for p in P]

def compute_rear_target(spot_origin, spot_width, spot_length, car_length,
                        inset=0.10, enter_from='bottom'):
    """Rear-bumper landing target on bay inner edge (centered in x)."""
    x = float(spot_origin[0]) + float(spot_width) / 2.0
    if enter_from == 'bottom':
        y = float(spot_origin[1]) + inset
    elif enter_from == 'top':
        y = float(spot_origin[1]) + float(spot_length) - inset
    else:
        raise ValueError("enter_from must be 'bottom' or 'top'")
    return np.array([x, y], dtype=float)

# -------------------- Path generation --------------------
def generate_path_for_strategy(strategy, car_length, car_width, spot_length, spot_width, distance_to_spot,
                               nA=45, nS=12, nB=28):
    """Local-frame template path; alignment happens later."""
    def clamp(v, lo, hi):
        return max(lo, min(hi, v))

    if strategy == "Forward Perpendicular":
        x_swing = clamp(0.4 * spot_width, 0.5, 1.4)
        approach_y = clamp(0.6 * distance_to_spot + 0.5 * spot_length,
                           1.2, distance_to_spot + spot_length - 0.4 * car_length)
        pathA = make_arc((0.0, 0.0), (+x_swing, 0.55 * approach_y),
                         (+0.25 * x_swing, approach_y), n=nA)
        pathS = straight_segment(tuple(pathA[-1]), length=0.3, heading_deg=85, n=nS)
        pathB = make_arc(tuple(pathS[-1]), (-0.45 * x_swing, 0.25 * spot_length),
                         (-0.18 * x_swing, 0.38 * spot_length), n=nB)
        return concat_paths(pathA, pathS, pathB)

    elif strategy == "Reverse Parallel":
        # --- Bezier S-curve reverse into bay (template space) ---
        def clamp(v, lo, hi):
            return max(lo, min(hi, v))

        # 1) Pull-ahead setup
        pull_ahead = clamp(0.8 * distance_to_spot, 0.5, max(0.6, distance_to_spot))
        path0 = straight_segment((0.0, 0.0), length=pull_ahead, heading_deg=85, n=max(8, nS))

        # 2) Target vehicle center just inside the bay inner edge (template y≈0)
        inset = 0.10
        rear_target_y = -inset              # inside line (downwards in template)
        target_center = (0.0, rear_target_y + 0.5 * car_length)

        # 3) Mid waypoint + lateral swing
        side_x = clamp(0.6 * spot_width + 0.4 * car_width, 0.9, 2.2)
        p_start = path0[-1]
        mid = (0.55 * p_start[0] + 0.45 * target_center[0],
               0.55 * p_start[1] + 0.45 * target_center[1] - 1.2)

        # 4) Control magnitudes and start tangent
        k1, k2 = 2.6, 1.8
        th = np.deg2rad(85.0)
        t0 = np.array([np.cos(th), np.sin(th)])

        # 5) Two cubic Beziers to shape S
        p0 = np.array(p_start)
        c1 = p0 - k1 * t0
        c2 = np.array([mid[0], mid[1] + 0.8])
        seg1 = sample_bezier(p0, c1, c2, np.array(mid), n=max(20, nA))

        c3 = np.array([mid[0], mid[1] - 0.8])
        c4 = np.array([target_center[0], target_center[1]]) + np.array([0.0, -k2])
        seg2 = sample_bezier(np.array(mid), c3, c4, np.array(target_center), n=max(20, nB))

        # 6) Impose lateral S (left then right)
        seg1 = [(x - 0.5 * side_x, y) for (x, y) in seg1]
        seg2 = [(x + 0.5 * side_x, y) for (x, y) in seg2]

        return concat_paths(path0, seg1[1:], seg2[1:])

    elif strategy == "Cannot Park":
        return None
    else:
        # fallback: simple forward arc
        p0 = (0.0, 0.0)
        p2 = (0.0, max(1.5, distance_to_spot + 0.6 * spot_length))
        p1 = (0.8 * spot_width, 0.5 * (p0[1] + p2[1]))
        return generate_bezier_curve(p0, p1, p2, n=max(60, nA + nB))

# -------------------- Plan & align --------------------
def plan_and_align(strategy, car_length, car_width, spot_length, spot_width, spot_origin,
                   distance_to_spot=1.0, inset=0.10, do_ghost_check=False):
    """Generate local template → auto-detect entry → rotate about current rear bumper → translate to rear_target."""
    full_path = generate_path_for_strategy(strategy, car_length, car_width, spot_length, spot_width,
                                           distance_to_spot=distance_to_spot, nA=60, nS=20, nB=40)
    if full_path is None:
        return {"curve_points": None, "rear_target": None, "yaw_target": None,
                "entry_side": None, "ghost_outline": None, "fits_width": None, "fits_length": None}
    curve_points = [tuple(pt) for pt in full_path]

    # Entry side detection
    spot_mid_y = spot_origin[1] + spot_length / 2.0
    start_y = curve_points[0][1]
    entry_side = 'bottom' if start_y < spot_mid_y else 'top'

    # Rear-bumper target on bay edge
    rear_target = compute_rear_target(spot_origin, spot_width, spot_length, car_length,
                                      inset=inset, enter_from=entry_side)

    # Rotate about current rear bumper to align yaw; for perpendicular bay, we want +y (90°)
    end_center = np.array(curve_points[-1])
    yaw_end    = heading_at(curve_points, len(curve_points)-1)
    rear_now   = rear_bumper_center(end_center, yaw_end, car_length)

    yaw_target = 90.0
    delta_yaw  = yaw_target - yaw_end
    curve_points = rotate_path_about_point(curve_points, rear_now, delta_yaw)

    # Translate so rear bumper lands exactly on the rear_target
    end_center_rot = np.array(curve_points[-1])
    yaw_end_rot    = heading_at(curve_points, len(curve_points)-1)
    rear_now_rot   = rear_bumper_center(end_center_rot, yaw_end_rot, car_length)

    delta = rear_target - rear_now_rot
    curve_points = np.array([(x + delta[0], y + delta[1]) for (x, y) in curve_points])

    return {
        "curve_points": curve_points,
        "rear_target": rear_target,
        "yaw_target": float(yaw_target),
        "entry_side": entry_side,
        "ghost_outline": None,
        "fits_width": None,
        "fits_length": None,
    }

# ================ Streamlit UI =================
run_button = st.button("Plan Path")

if run_button:
    plan = plan_and_align(
        strategy=strategy,
        car_length=car_length,
        car_width=car_width,
        spot_length=spot_length,
        spot_width=spot_width,
        spot_origin=spot_origin,
        distance_to_spot=distance_to_spot,
        inset=0.10,
        do_ghost_check=False
    )

    curve_points = plan["curve_points"]
    rear_target  = plan["rear_target"]

    if curve_points is None:
        st.error("No feasible path returned.")
    else:
        # Plot
        fig, ax = plt.subplots(figsize=(6,5))
        ax.grid(True)

        # Parking bay rectangle
        spot_rect = patches.Rectangle((spot_origin[0], spot_origin[1]),
                                      spot_width, spot_length,
                                      linewidth=1.5, edgecolor='green', facecolor='none',
                                      linestyle='--', label='Parking Spot', zorder=5)
        ax.add_patch(spot_rect)

        # Rear target marker
        ax.scatter(rear_target[0], rear_target[1], s=80, marker='x', label='Rear Target')

        # Path
        ax.plot(curve_points[:,0], curve_points[:,1], lw=2, label='Path')

        # Limits & aspect
        xs_path = curve_points[:,0]; ys_path = curve_points[:,1]
        sx0, sy0 = float(spot_origin[0]), float(spot_origin[1])
        sx1, sy1 = sx0 + float(spot_width), sy0 + float(spot_length)
        pad = 0.8
        ax.set_xlim(min(xs_path.min(), sx0, sx1) - pad, max(xs_path.max(), sx0, sx1) + pad)
        ax.set_ylim(min(ys_path.min(), sy0, sy1) - pad, max(ys_path.max(), sx0, sy0) + pad)
        ax.set_aspect('equal', adjustable='box')
        ax.legend(loc='upper left')

        st.pyplot(fig)

        # Summary
        st.subheader("Plan Summary")
        st.write(f"- Strategy: **{strategy}**")
        st.write(f"- Entry side (auto): **{plan['entry_side']}**")
        st.write(f"- Car (L×W): **{car_length:.2f} m × {car_width:.2f} m**")
        st.write(f"- Spot (L×W): **{spot_length:.2f} m × {spot_width:.2f} m**")
        st.write(f"- Distance to spot: **{distance_to_spot:.2f} m**")
else:
    st.info("Set inputs and click **Plan Path**.")

Overwriting app.py


In [4]:
!pip install pyngrok
from pyngrok import ngrok
from getpass import getpass

ngrok_token = getpass("Enter your ngrok token: ")
!ngrok config add-authtoken $ngrok_token

Enter your ngrok token: ··········
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
# Get a public link via ngrok
from pyngrok import ngrok

# Kill any previous tunnels
ngrok.kill()

# Run Streamlit in the background
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)

!streamlit run app.py --server.headless true --server.port 8501 &

Streamlit URL: NgrokTunnel: "https://cad47e076d2d.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.147.52.36:8501

  Stopping...
